In [1]:
# DL needs
import tensorflow as tf
import keras as kr

# Data needs
import pandas as pd
from sklearn.model_selection import train_test_split

# Numerical computation needs
import numpy as np

# plotting needs
import matplotlib.pyplot as plt
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

# ensuring reproducibility
random_seed=42
tf.random.set_seed(random_seed)

import sys
sys.path.append('/home/rudraksha14/Desktop/RAY_RISE_ABOVE_YOURSELF/Programming/tensorflow/')
import important_functionalities as impf

2025-03-12 18:02:44.681722: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# load data
data_dir='/home/rudraksha14/Desktop/RAY_RISE_ABOVE_YOURSELF/Programming/tensorflow/6_transfer_learning_feature_extraction/10_food_classes_10_percent'
train_dir=data_dir+'/train'
valid_dir=data_dir+'/test'

In [3]:
IMAGE_SHAPE=(224,224)
BATCH_SIZE=32
train_data=tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                               image_size=IMAGE_SHAPE,
                                                               label_mode='categorical',
                                                                batch_size=BATCH_SIZE
                                                               )
valid_data=tf.keras.preprocessing.image_dataset_from_directory(directory=valid_dir,
                                                               image_size=IMAGE_SHAPE,
                                                                label_mode='categorical',
                                                                 batch_size=BATCH_SIZE
                                                                )

Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


| Experiment | Data       | Preprocessing | Model          |
|------------|------------|---------------|----------------|
| Model 0<br>(baseline)     | 10 classes of food101 dataset (random 10% training data only)  | None        | Feature Extractor: EfficientNetB0 (pretrained on ImageNet, all layers frozen) with no top|

In [5]:
# Model 0: Building a transfer learning model using Keras Functional API

### 1. Create the base model with tf.keras.applications
base_model=tf.keras.applications.efficientnet_v2.EfficientNetV2B0(include_top=False,#exclude o/p layer
                                                                  weights='imagenet',
                                                                 input_shape=(224,224,3)) 

### 2. Freeze the layers in the base model
base_model.trainable=False

### 3. Create inputs into our model
inputs = tf.keras.layers.Input(shape=(224,224,3),name='input_layer')    

### 4. If using a model like ResNet50V2 you will need to normalize inputs (you don't have to for EfficientNet)
# x = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)(inputs)

### 5. Pass the inputs to the base_model
x = base_model(inputs)
print(f"Shape after base_model: {x.shape}")

### 6. max pool the outputs of the base model (aggregate all the most important information, reduce number of computations)
x = tf.keras.layers.GlobalmaxPooling2D(name='global_max_pooling_layer')(x)
print(f"After GlobalmaxPooling2D(): {x.shape}")

### 7. Create the output activation layer
outputs = tf.keras.layers.Dense(10,activation='softmax',name='output_layer')(x)

### 8. Combine the inputs with the outputs into a model
model_0=tf.keras.Model(inputs,outputs)


### 9. Compile the model
model_0.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

### 10. Fit the model and save its history
history_10_percent=model_0.fit(train_data,
                               epochs=5,
                               steps_per_epoch=len(train_data),
                               validation_data=valid_data,
                               validation_steps=int(0.25*len(valid_data)),
                               callbacks=[impf.create_tensorboard_callback(dir_name="transfer_learning",     experiment_name="10_percent_feature_extraction")])

Shape after base_model: (None, 7, 7, 1280)
After GlobalAveragePooling2D(): (None, 1280)
Saving TensorBoard log files to : transfer_learning/10_percent_feature_extraction/20250312-180427
Epoch 1/5


2025-03-12 18:04:35.156949: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 51380224 exceeds 10% of free system memory.
2025-03-12 18:04:35.217100: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 51380224 exceeds 10% of free system memory.
2025-03-12 18:04:35.318741: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 51380224 exceeds 10% of free system memory.
2025-03-12 18:04:35.361255: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 51380224 exceeds 10% of free system memory.


 1/24 ━━━━━━━━━━━━━━━━━━━━ 3:15 8s/step - accuracy: 0.1250 - loss: 2.4220

2025-03-12 18:04:36.071281: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 51380224 exceeds 10% of free system memory.


24/24 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 0.2744 - loss: 2.1213 - val_accuracy: 0.7368 - val_loss: 1.3092
Epoch 2/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0.7194 - loss: 1.2431 - val_accuracy: 0.8306 - val_loss: 0.8809
Epoch 3/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - accuracy: 0.8149 - loss: 0.8399 - val_accuracy: 0.8421 - val_loss: 0.7113
Epoch 4/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - accuracy: 0.8448 - loss: 0.6997 - val_accuracy: 0.8470 - val_loss: 0.6304
Epoch 5/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - accuracy: 0.8760 - loss: 0.5899 - val_accuracy: 0.8536 - val_loss: 0.5657


In [6]:
# evaluate on complete validation data
model_0.evaluate(valid_data)

79/79 ━━━━━━━━━━━━━━━━━━━━ 55s 694ms/step - accuracy: 0.8490 - loss: 0.5770


[0.579102635383606, 0.8507999777793884]

**2.2. Getting feature vector from trained model**

* To demonstrate the Global max Pooling 2D layer 
  * We have a tensor after our model goes through `base_model` of shape (None,7,7,1280) ....
  * When we pass this through Global max Pooling 2D layer, it turns into (None,1280)
  * Let's use a similar shape tensor of (1,4,4,3) and then pass it through GlobalmaxPooling2D {transforms a 4D tensor into a 2D tensor}
* A feature vector is a learned representation of the input data (a compressed form of the input data based on how the model sees it)

In [31]:
input_shape=(1,4,4,3)

# create a random tensor
input_tensor=tf.random.normal(input_shape)
print(f"Random input tensor:\n {input_tensor}\n")

# pass the random tensor through a global max pooling 2D layer
global_max_pooled_tensor=tf.keras.layers.GlobalmaxPooling2D()(input_tensor)
print(f"2D global max pooled random tensor:\n {global_max_pooled_tensor}\n")

# check the shape
print(f"Shape of input tensor: {input_tensor.shape}")
print(f"Shape of 2D global max pooled random tensor: {global_max_pooled_tensor.shape}")

Random input tensor:
 [[[[-0.23585585  0.33401316  0.6599627 ]
   [ 1.1498349   0.80851334  1.9857812 ]
   [ 0.24859004  0.6843617  -1.4259181 ]
   [ 1.4124185   0.9517439   1.7974569 ]]

  [[-0.37729186  0.3015694   0.4366594 ]
   [ 1.2700577   0.326862   -0.4133045 ]
   [ 1.1873597  -0.5737731  -0.17353044]
   [-0.5922727  -0.27917758  0.89432085]]

  [[ 0.4163104  -0.23119776  0.20691264]
   [ 1.5541818  -1.1967332  -1.6391214 ]
   [ 0.15168859  1.4021907  -1.5639578 ]
   [ 1.4441936  -0.17168391 -0.4327975 ]]

  [[ 0.93174577 -0.077992   -0.56287485]
   [ 0.6456059  -0.60841733  0.7858203 ]
   [-0.77531946  0.68973136 -0.56853956]
   [-0.445213   -1.2932749   1.0399288 ]]]]

2D global average pooled random tensor:
 [[0.49912712 0.06667098 0.06417489]]

Shape of input tensor: (1, 4, 4, 3)
Shape of 2D global average pooled random tensor: (1, 3)


In [32]:
# replicating global max pool 2D
tf.reduce_mean(input_tensor,axis=[1,2])

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[0.49912712, 0.06667098, 0.06417489]], dtype=float32)>

In [33]:
input_shape=(1,4,4,3)

# create a random tensor
input_tensor=tf.random.normal(input_shape)
print(f"Random input tensor:\n {input_tensor}\n")

# pass the random tensor through a global max pooling 2D layer
global_max_pooled_tensor=tf.keras.layers.GlobalMaxPooling2D()(input_tensor)
print(f"2D global max pooled random tensor:\n {global_max_pooled_tensor}\n")

# check the shape
print(f"Shape of input tensor: {input_tensor.shape}")
print(f"Shape of 2D global max pooled random tensor: {global_max_pooled_tensor.shape}")

Random input tensor:
 [[[[ 0.6221494   0.39071774  0.572821  ]
   [ 0.92682797  0.8460992   0.08634651]
   [-0.39511812 -0.02012417  1.0490924 ]
   [-0.3120731   0.41652173  0.8515276 ]]

  [[-1.27271    -0.09542792 -0.16090107]
   [-0.900317    0.9836345   0.94048667]
   [ 1.4384675  -0.16801515 -0.43124875]
   [-1.1060885   0.2186191   1.4111438 ]]

  [[ 0.826967    1.1009964   0.02034463]
   [-0.36579552  0.10911851 -0.22597931]
   [ 0.3696448  -0.8651293   1.2661022 ]
   [-0.49262673 -0.99944526 -0.08006851]]

  [[ 1.7055075   0.10260425  1.1992584 ]
   [-0.13475318 -0.4856558  -1.5140239 ]
   [ 0.2532031   1.3699163   2.1776462 ]
   [ 1.8093497  -1.3124373   0.39335868]]]]

2D global max pooled random tensor:
 [[1.8093497 1.3699163 2.1776462]]

Shape of input tensor: (1, 4, 4, 3)
Shape of 2D global max pooled random tensor: (1, 3)


In [34]:
# replicating global max pool 2D
tf.reduce_max(input_tensor,axis=[1,2])

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[1.8093497, 1.3699163, 2.1776462]], dtype=float32)>

***-- CONTD IN NEXT NOTEBOOK --***